# Overview

## Environment
Implements the physical properties between the robot body and the environment in which it evolves (can be real or simulated) 
    - Each environment implements its own `compute_sensori_effect` function which takes motor command vector $m$ and returns the corresponding sensory effect vector $s$
    - An environment config  provides information about the motor and sensori ranges used by the environment 
   

## Sensorimotor Model
Learns mappings between robot motor actions and the sensory effect they produce. Implements both:
    1. The **iterative learning process** from sensorimotor experience, i.e. from the iterative collection of  (𝑚,𝑠) pairs by interaction with the environment
    2. The use of the resulting internal model to perform **forward and inverse predictions** (or any kind of general prediction between sensorimotor subspaces).

As a result, every Sensorimotor Model will be able to: 
- Infer the \<sensory result\> from a given motor command (forward prediciton)
- Infer the motor command allowing to reach a particular \<sensory result\> (inverse prediction)
- Update online from sensorimotor experience


Explauto implements various sensorimotor models: Simple nearest-neighbor look-up, non-parametric models combining classical regressions and optimization algorithms, online local mixtures of Gaussians (beta)

    
## Interest Space
Areas of exploration from which goals are sampled. The interest space can be either;
- Motor space: Results in **motor babbling** strategies
- Sensory space: Results in **goal babbling** strategies


## Interest Model
Implements the active exploration process (curiosity). Explores the interest space (either motor or sensory) by sampling goals using a [**sampling procedure**](#Sampling-Procedures). 

## Agent
Encapsulates sensorimotor and interest models
- Allows to generalize and simplify the simulation loop
- Removes bootstrapping issues encountered when training a sensorimotor model while also needing initial inverse prediction(s)


## High-Level Example

An example of motor babbling implementation without using an Agent (to clearly show the steps): 

```python
# Instantiate an interest model
im_model = InterestModel.from_configuration(environment_conf=, interest_space_dimensions=, interest_model_name=, config_name=)
for _ in range(100):
    # sample a sensory goal maximizing learning progress using the interest model:
    motor_goal = im_model.sample()
    # use a trained sensorimodel to make a forward prediction from the motor goal to expected sensory effect:
    # Note: Agent would handle initial bootstrapping on sensorimodel
    sensori_inferred = sm_model.forward_prediction(motor_goal)
    # execute the command and observe the corresponding sensory effect:
    sensori_actual = environment.compute_sensori_effect(motor_goal)
    # update the sensorimodel:
    sm_model.update(motor_goal, sensori_actual)
    # update the interestmodel:
    im_model.update(np.hstack((motor_goal, sensori_actual)), np.hstack((motor_goal, sensori_inferred)))
```

# Sensorimotor Models

- **Forward Models**: Learn relationship of motor controllers to sensory effects
- **Inverse Models**: Learn relationship of sensory effects (goals) to the motor program/action allowing to reach them

* `xy` -> motor order/sensory effect pair to the model
    - :arg x:  an input (order) vector compatible with self.Mfeats.
    - :arg y:  a output (effect) vector compatible with self.Sfeats.
    
* `m`: Motor part of `x,y`
* `s`: Sensory part of `x,y`

## Online Learning Algorithms (Models)
Trained iteratively during the interaction of the robot in the environment in which it evolves.

### Non-Parametric 
Combining classical regressions and optimization algorithms

Consists of the following:
- **dataset**: Stores all the experiments ($m$, $s$) into a list
- **forward model**: Uses the dataset for the forward prediction
- **inverse model**: Uses the forward model _or_ the dataset directly to perform inverse prediction

Two operating modes:
- explore: When the agent asks for the exact inverse prediction $m$ of a goal $s_g$, $m$ will be perurbated with some gaussian exploration noise in order to allow the agent to explore new motor commands.
    - As a result, the sensormitor models have a common parameter `sigma_explo_ratio` (default = 0.1) which is the standard deviation of the gausssian noise scaled depending on the motor domain size. If a motor value is bounded in [-2:2], then a `sigma_explo_ratio` of 0.1 will induce an exploration noise of (`m_max` - `m_min`) * `sigma_explo_ratio`). 0.4 in this example. 
- exploit: No exploration noise is added. This mode is used for instance when evaluating the inverse model for comparison purposes.


Can use combinations of forward and inverse models -- there are some provided or can define your own. For example, `LWLR-CMAES` will use a forward model of `LWLR` and an inverse model `CMAES`


#### Forward Models

Predict $s_p$ given a $m$ that might have never been observed, using the dataset of observations ($m$, $s$)

##### NN (Nearest Neighbor)

Works sufficiently well in different typical robotic applications. 

To perform a forward prediction the Nearest Neighbor model looks in the dataset of tuples ($m$, $s$) for the nearest neighbor of the given $m$ motor command, and returns it's corresponding $s$. This forward model is very fast (up to datasets of size $10^5$) and makes no assumptions about hte regularity of the model being learned (continuity, linearity, ...). 

##### NSNN (Non-Stationary Nearest Neighbor)
Modified version of NN for non-stationary environments.

Points are not only weighted by distance but also by the number of points that appeared after that one (gaussian with parameter `sigma_t`=100), to put less weight on old points and allow the learning of non-stationary environments.


##### WNN (Weighted Nearest Neighbor)

To perform a forward prediction of $m$, the Weighted Nearest Neighbor model looks at the $k$ nearest neighbors of $m$ in the dataset and returns the average of the $k$ corresponding $s$. This average is weighted by the distance to $m$ with a gaussian of standard deviation $\sigma$

##### ES-WNN

WNN with estimated sigma, on a query basis, as the mean distance.

##### LWLR (Locally Weighted Linear Regression)

Computes a linear regression of the $k$ nearest neighbors of $m$ (thus a local regression) and finds the requested $s$ with the given $m$ based on that regression. 

References :
1. https://en.wikipedia.org/wiki/Local_regression
2. C. G. Atkeson, A. W. Moore, S. Schaal, "[Locally Weighted Learning for Control](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.468.7121&rep=rep1&type=pdf)", "Springer Netherlands", 75-117, vol 11, issue 1, 1997/02, 10.1023/A:1006511328852    
3. See also a [video](http://www.cosmolearning.com/video-lectures/locally-weighted-regression-probabilistic-interpretation-logistic-regression/) lecture on LWR.


Pseudo Code: 
```python
Input D problem dimension
Input X matrix of inputs:  X[k][i] = i’th component of k’th input point.
Input Y matrix of outputs: Y[k] = k’th output value.
Input xq = query input.    Input kwidth.

WXTWX = empty (D+1) x (D+1) matrix
WXTWY = empty (D+1) x 1     matrix

for ( k = 0 ; i <= N - 1 ; i = i + 1 )
    # Compute weight of kth point
    wk = weight_function( distance( xq , X[k] ) / kwidth )

    /* Add to (WX) ^T (WX) matrix */
    for ( i = 0 ; i <= D ; i = i + 1 )
        for ( j = 0 ; j <= D ; j = j + 1 )
            if ( i == 0 )
                xki = 1 else xki = X[k] [i]
            if ( j == 0 )
                xkj = 1 else xkj = X[k] [j]
            WXTWX [i] [j] = WXTWX [i] [j] + wk * wk * xki * xkj

    /*  Add to (WX) ^T (WY) vector */
    for ( i = 0 ; i <= D ; i = i + 1 )
        if ( i == 0 )
            xki = 1 else xki = X[k] [i]
        WXTWY [i] = WXTWY [i] + wk * wk * xki * Y[k]

/* Compute the local beta.  Call your favorite linear equation solver.
   Recommend Cholesky Decomposition for speed.
   Recommend Singular Val Decomp for Robustness. */

Beta = (WXTWX)^{-1}(WXTWY)

Output ypredict = beta[0] + beta[1]*xq[1] + beta[2]*xq[2] + … beta[D]*x q[D]
```

##### NSLWLR (Non-Stationary Locally Weighted Linear Regression)
Modified version of LWLR for non-stationary environments.

Points are not only weighted by distance but also by the number of points that appeared after that one (gaussian with parameter `sigma_t`=100), to put less weight on old points and allow the learning of non-stationary environments.

##### ES-LWLR

LWLR with estimated sigma, on a query basis, as the mean distance

#### Inverse Models

Inverse models infer a motor command $m$ that should be able to reach a given goal $s_g$

##### NN (Nearest Neighbor)

To perform the inverse inference, the Nearest Neighbor inverse model looks in the dataset of tuples $(m, s)$ for the nearest neighbor of the given $s$ motor command, and return its corresponding $m$.

##### NSNN (Non-Stationary Nearest Neighbor)
Modified version of NN for non-stationary environments.

Points are not only weighted by distance but also by the number of points that appeared after that one (gaussian with parameter `sigma_t`=100), to put less weight on old points and allow the learning of non-stationary environments.


##### WNN (Weighted Nearest Neighbor)

Typical robotic forward models are very redundant. For example, a robotic arm can move it's hand to position $s$ with infinite possible $m$ motor positions.

As a result, when trying to infer a motor command $m$ to reach a given goal $s$ an average of the nearest neighbors of $s$ in the dataset would make no sense as those nearest neighbors might have very different corresponding motor commands.

To perform the inverse inference of a given $s$, the Weighted Nearest Neighbor model looks at the nearest neighbor of $s$ in the dataset and gets its corresponding $m$. It then finds the $k$ (parameter) nearest neighbors of $m$ in the dataset, and returns their average weighted by the distance of their sensory part to $s$, with a gaussian of standard deviation $\sigma$ (parameter).

See code [here](https://github.com/flowersteam/explauto/blob/master/explauto/sensorimotor_model/inverse/wnn.py#L25).

##### ES-WNN

WNN with estimated sigma, on a query basis, as the mean distance.

##### Jacobian

TODO: find more details on their implementation

##### COBYLA (Constrained Optimization BY Linear Approximation) 

**Optimization algorithm** to minimize the error $e(x) = ||f(x) - y_g||^2$  where $y_g$ is the goal, $f$ is the forward model, and $x$ is the motor command to be inferred.

A numerical optimization method for constrained problems where the derivative of the objective function is not known

https://docs.scipy.org/doc/scipy/reference/optimize.minimize-cobyla.html

COBYLA specific param:
* `maxiter`: Limits the number of error function (and as a result, forward model) evaluation.

##### BFGS (Broyden-Fletcher-Goldfarb-Shanno Optimization Algorithm)

_**Optimization algorithm** to minimize the error $e(x) = ||f(x) - y_g||^2$  where $y_g$ is the goal, $f$ is the forward model, and $x$ is the motor command to be inferred._

https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html

BFGS specific param:
* `maxfun`: Limits the number of error function (and as a result, forward model) evaluation.

##### L-BFGS-B (Limited Memory Bounded Broyden-Fletcher-Goldfarb-Shanno Optimization Algorithm)

**Optimization algorithm** to minimize the error $e(x) = ||f(x) - y_g||^2$  where $y_g$ is the goal, $f$ is the forward model, and $x$ is the motor command to be inferred.

A limited-memory quasi-Newton code for bound-constrained optimization

https://en.wikipedia.org/wiki/Limited-memory_BFGS  
http://users.iems.northwestern.edu/~nocedal/lbfgsb.html  
https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html

BFGS specific param:
* `maxfun`: Limits the number of error function (and as a result, forward model) evaluation.

##### CMA-ES (Covariance Matrix Adaptation - Evolutionary Strategy)

**Optimization algorithm** to minimize the error $e(x) = ||f(x) - y_g||^2$  where $y_g$ is the goal, $f$ is the forward model, and $x$ is the motor command to be inferred.

Inverse model also optimizes the error function above but makes fewer assumptions on the regularity of the forward model to perform the search. It is based on a random exploration (with a computed covariance) around a current point of interest, and adapts this point and recompute the covariance matrix at each iteration, with memory of the taken path.

The initial point is set as the motor part $m$ of the nearest neighbor $s$ of the goal $s_g$, and the initial covariance matrix is identity times an exploration $\sigma$ (parameter). This inverse model also takes a 'maxfevals' parameter that limits the number of forward model evaluations.

See [Hansen's website](http://www.cmap.polytechnique.fr/~nikolaus.hansen/) and this [tutorial](https://arxiv.org/abs/1604.00772) on CMA-ES.

### IMLE

**⚠️** **Appears unused**

IMLE model from Bruno Damas

> We present a supervised learning algorithm for estimation of generic input-output relations in a real-time, online fashion. The proposed method is based on a generalized expectation-maximization approach to fit an infinite mixture of linear experts (IMLE) to an online stream of data samples. This probabilistic model, while not fully Bayesian, can efficiently choose the number of experts that are allocated to the mixture, this way effectively controlling the complexity of the resulting model. The result is an incremental, online, and localized learning algorithm that performs nonlinear, multivariate regression on multivariate outputs by approximating the target function by a linear relation within each expert input domain and that can allocate new experts as needed. A distinctive feature of the proposed method is the ability to learn multivalued functions: one-to-many mappings that naturally arise in some robotic and computer vision learning domains, using an approach based on a Bayesian generative model for the predictions provided by each of the mixture experts. As a consequence, it is able to directly provide forward and inverse relations from the same learned mixture model. We conduct an extensive set of experiments to evaluate the proposed algorithm performance, and the results show that it can outperform state-of-the-art online function approximation algorithms in single-valued regression, while demonstrating good estimation capabilities in a multivalued function approximation context.

More details:

- https://direct.mit.edu/neco/article-abstract/25/11/3044/7931/Online-Learning-of-Single-and-Multivalued
- https://github.com/bdamas/IMLE
- http://users.isr.ist.utl.pt/~bdamas/IMLE (Note! I have not tried downloading the zip...)

### ~ILO GMM~

**⚠️** **Appears unused**

### ~Bayesian Optimisation~

**⚠️** **Appears unused**

### Discrete (LidstoneModel)

**⚠️** **Appears to have only been used with DiscretizedProgress**

# Interest Models

## Competence

At each iteration:
1. A goal is selected by the interest model 
2. The sensorimotor model tries to reach that goal 
3. The **competence** of the goal is computed

There are three ways to determine the **competence** of a goal:
* **Distance**: The distance between the actual reached point and the goal
    - Used by Discretized sampling procedure
* **Exponential**:  $e^{-power\times||g-s||}$
    - Used by Tree sampling procedur
* **Bool**: Return goal == reached
  


## Sampling Procedures

Sample the interest space. For non-random sampling procedures, the sampled goal should serve to improve the prediction of the sensorimotor model by maximizing learning progress. Estimates how a given action is useful for learning and samples the best ones

**Sensorimotor Experiment**: The selection of a sample with the purpose of improving the sensorimotor model

### Random Progress

Uniformly sample the choice space ranges // draws random goals in the interest space. 


### Discretized Progress
Divides sensorimotor choice space into a grid of dimensions $sensorimotor\_mins \times sensorimotor\_maxs$ and maintains an empirical measure of the learning progress in each cell. Samples a cell according to the **learning progress** (favoring cells displaying high progresses) and samples a random point in that chosen cell. Each cell keeps a history of the recent learning errors observed.

**⚠️** If the number of sensorimotor dimensions is large (>2), the _discretization progress procedure _won't be feasible_ as the number of regions is exponential in the number of dimensions. 

- $x$: A sample point in the choice space $X$ (`expl_dims`) (i.e. a choice)
    - Motor Babbling: $X$ corresponds to the motor space $M$ 
    - Goal Babbling: $X$ corresponds to sensory space $S$   
<br>
<br>
-  $y$: Prediction performed by the sensorimotor model for the choice $x$ in space $Y$ (`inf_dims`)
    - Motor Babbling: A **forward** prediction where $Y=S$
    - Goal Babbling: An **inverse** prediction where $Y=M$ 
<br>
<br>
- $xy$: Concatenation of $x$ and $y$ reordered as a vector in $M \times S$

- $m$: The executed motor command
- $s$: The observed sensory consequence 

- `x_card`: The total numbers of cells in the discretization
- `win_size`: window size of the interest computation which is based on the last `win_size` points

- **Learning Error**: The distance between $xy$ and $ms$ 
- **Learning Progress**: The opposite of covariance between time (relative to a particular cell) and **learning error** (i.e. the agent is progressing in that cell if the covariance between time and error is negative). 

### Tree Progress



Adapts the discretization to the dataset distribution. At each iteration, if there are too many points in a region, that region is split into two subregions (along the next axis in a kdtree-like way). The value of the split is chosen to best discriminate the interest of the 2 subregions.


* `max_points_per_region`:  Maximum number of points per region. A given region is split when this number is exceeded.
* `max_depth`: Maximum depth of the tree
* `split_mode`: Mode to split a region: 
    - `random`: random value between first and last points
    - `median`: median of the points in the region on the split dimension
    - `middle`: middle of the region on the split dimension
    - `best_interest_diff`: value that maximize the difference of progress in the 2 sub-regions (SAGG-RIAC)
* `progress_win_size`: Number of last points taken into account for progress computation (should be < `max_points_per_region`)
* `progress_measure` How to compute progress: 
    - `abs_deriv_cov`: Approach from the discrete progress interest model 
    - `abs_deriv`: absolute difference between first and last points in the window
    - `abs_deriv_smooth`: absolute difference between first and last half of the window
* `sampling_mode`: How to sample a point in the tree: 
    - `dict(multiscale=bool, volume=bool, mode='greedy'|'random'|'epsilon_greedy'|'softmax', param=float)`
        - `multiscale`: if we choose between all the nodes of the tree to sample a goal, leading to a multi-scale resolution (SAGG-RIAC)
        - `volume`: if we weight the progress of nodes with their volume to choose between them (used by `random` and `softmax` sampling modes)
        - `mode`
            - `greedy`: Sample a point in the leaf with the max progres
            - `random`: Sample a point in a random leaf
            - `epsilon_greedy`: Sample a point in the leaf with the max progress with probability (1-`eps`) and a random leaf with probability (`eps`)
                - `epsilon`
            - `softmax`:  Sample leaves with probabilities progress*volume and a softmax exploration (with a temperature parameter)
                - `temperature`

#### [IAC](https://www.cs.swarthmore.edu/~meeden/DevelopmentalRobotics/iac07.pdf) (2007)

I think the playground experiment was using some early iterations of this Tree implementation. 

#### [R-RIAC](http://www.pyoudeyer.com/TAMDBaranesOudeyer09.pdf) (2009)
TODO: read paper and populate this

#### [SAGG_RIAC](http://www.pyoudeyer.com/ActiveGoalExploration-RAS-2013.pdf) (2013)
TODO: read paper and populate this more
* $card(r_i)$: The number of points in the sub-region $i$
* $progress(r_i)$: The absolute derivative of the competence on the points of the sub-region $r_i$
* **Competence** on goal point $g$ with observed sensori consequence $s$: $e^{-power\times||g-s||}$ with $power=10$

### Gaussian Mixture Model Progress

Related paper [here](https://flowers.inria.fr/FrontierscogSciJul13.pdf)

TODO: Look into this more?

Computes a gaussian mixture model that represents (simultaneously) the space of interest, the competence, and time (thus, a mixture in $S \times C \times T$ space). To sample an interesting region of $S$, the algoirthm weights the gaussian components based on their covariance between $C$ and $T$, giving positive weight to a component if the competence increases with time in that region of $S$

# Context Mode


_Note: I didn't dive into this much since I'm more interested in other interest models. And this seems to be captures by those?_


Context mode can used with Random and DiscretizedProgress Interest models. If selected, will also impact Sensorimotor model. 

Params: 
* `mode` 
    - 'None' | 'mcs' | 'mdmsds'

## None

Seems to be the behaviour for all other Interest models. 

## mcs 
Allow the learning and control of actions that depend on the context provided by the environment (only sensory context). More details here -> learning_with_environment_context.ipynb

> The sensory feedback is now the concatenation of the context  $c$ and the feedback of the arm  $s$. We thus call this mode "mcs".


To draw a goal given a context, the interest model has to be 'RandomInterest' or 'DiscretizedProgress':

* If RandomInterest model, will draw a random goal in the dimensions of $s$ (not $c$). i.e. will sample randomly on dimensions not in context. 
* If DiscretizedProgress model, will draw a goal in the $s$ region where the progress is maximal on points when the context was similar to $c$. i.e. Samples the region with max progress among regions that have the same context. 


MCS Specific Params:
* `reset_iterations`: Number of iterations before the environment is reset
* `context_n_dims`: how many dimensions the context has
* `context_sensory_bounds`: specify the min and max bounds on those dimenions

## mdmsds 
Define local actions that depend on the previous motor and sensory positions. Allow the learning and control of local actions that depend on a sensory and motor context. 
More details here -> learning_with_sensorimotor_context.ipynb


MDMSDS Specific Params:
* `choose_m`
* `rest_positions`
* `dm_bounds`
* `ds_bounds`

# Things to Revisit

* https://nbviewer.org/github/sebastien-forestier/ExplorationAlgorithms/blob/master/main.ipynb

TODO:
- ball tree (cone tree?) with cos-sim clustering.
- Read 'Exploration strategies in developmental robotics: a unified probabilistic framework'
- Look into Interest Model GMM